In [ ]:
from hanlp_restful import HanLPClient
import json
import os
import time

HanLP = HanLPClient('https://www.hanlp.com/api', auth="", language='zh')
HanLP.parse(["我是李刚", "我爸是李刚"], tasks='pos/ctb')

{'tok/fine': [['我', '是', '李刚'], ['我', '爸', '是', '李刚']],
 'pos/ctb': [['PN', 'VC', 'NR'], ['PN', 'NN', 'VC', 'NR']]}

In [3]:
def plot_token_freq(tokens, topk=100):
    from collections import Counter
    matplotlib.rc("font",family='YouYuan')
    counter = Counter(tokens)
    temp = list(counter.items())
    temp.sort(key=lambda var: var[1], reverse=True)
    temp = temp[:topk]
#     plt.bar([var[0] for var in temp], [var[1] for var in temp])
#     plt.show()
    print(temp)

In [4]:
def split_into_blocks(text, segment_size=150):
    # split text into a list of segments
    segment_list = []
    i = -1
    while True:
        i += 1
        left, right = i*segment_size, (i+1)*segment_size
        segment = text[left: right]
        segment_list.append(segment)
        if right >= len(text):
            break
    return segment_list

In [8]:
block_size = 150
big_block_size = 14000

results_dict = dict()

In [10]:
def join(ll):
    return_l = []
    for l in ll:
        return_l = return_l + l
    return return_l

In [ ]:
texts = []
query_timestamps = []
root = "all_cre1"
for file_name in os.listdir(root):
    print("currently processing", file_name)
    if file_name in results_dict:
        continue
    with open(os.path.join(root, file_name), 'r', encoding='utf-8') as rf:
        text = rf.read()
#         def read_in_blocks(text, block_size=15000):
#             while True:
#                 block = text.read(block_size)
#             if not block:
#                 return  # 文件读取完毕
#         read_in_blocks(text)
    
    all_tokens = []
    all_pos_tags = []
    
    big_block_list = split_into_blocks(text, 14000)
    
    for big_block in big_block_list:
        block_list = split_into_blocks(big_block, 150)

        if (len(query_timestamps) < 60):
            gap = 10000
        else:
            current_time = time.time()
            gap = current_time - query_timestamps[-60]
        if gap < 70:
            print("\tsleeping for", 70-gap)
            time.sleep(70-gap)
        query_timestamps.append(time.time())
        
        # 分词
        data = HanLP.parse(block_list, tasks='pos/ctb')
        tokens = data['tok/fine']
        pos_tags = data['pos/ctb']

        all_tokens = all_tokens + join(tokens)
        all_pos_tags = all_pos_tags + join(pos_tags)

    results_dict[file_name] = {"all_tokens": all_tokens, "all_pos_tags": all_pos_tags}
#     print (all_tokens)

In [ ]:
import json

#with open("results_dict_all_cre.json", "w", encoding='utf-8') as f:
    #json.dump(results_dict, f)
with open("results_dict_all_cre.json", "r", encoding='utf-8') as f:
     results_dict = json.load(f)

In [14]:
for file_name in results_dict:
    all_tokens, all_pos_tags = results_dict[file_name]['all_tokens'], results_dict[file_name]['all_pos_tags']
    noun_tokens = []
    for i in range(len(all_pos_tags)):
        if all_pos_tags[i].startswith("N"):
            noun_tokens.append(all_tokens[i])
    results_dict[file_name]["noun_tokens"] = noun_tokens

In [16]:
def merge_stopwords(*stopwords_lists):
    merged_stopwords = set()

    for stopwords_file in stopwords_lists:
        with open(stopwords_file, 'r', encoding='utf-8') as file:
            stopwords = set(file.read().split())
            merged_stopwords.update(stopwords)

    return list(merged_stopwords)

stopwords_list1 = 'scu_stopwords.txt'
stopwords_list2 = 'cn_stopwords.txt'
stopwords_list3 = 'baidu_stopwords.txt'
stopwords_list4 = 'hit_stopwords_cre.txt'


merged_stopwords = merge_stopwords(stopwords_list1, stopwords_list2, stopwords_list3, stopwords_list4)

In [18]:
for filename in results_dict:
    token_list = results_dict[filename]['noun_tokens']
    all_words = [word for word in token_list if word not in merged_stopwords]
    results_dict[filename]['cleaned_noun_tokens'] = all_words
#     print(all_words)

In [19]:
texts = []
numbers = []
textid2name = dict()
for key, item in results_dict.items():
    texts.append(item['cleaned_noun_tokens'])
    textid2name[len(texts)-1] = key
    numbers.append(len(item['cleaned_noun_tokens']))
numbers.sort()
print(numbers)

def split_list(l, length=180):
    target = len(l)//length + 1
    segment_length = round(len(l)/target)
    return split_into_blocks(l, segment_length)
new_texts = []

chunk2text = dict()
for i in range(len(texts)):
    text = texts[i]
    splitted_text = split_list(text)
    for j in range(len(splitted_text)):
        new_texts.append(splitted_text[j])
        chunk2text[len(new_texts)-1] = {"id of original doc": i, "name of original doc": textid2name[i], "chunk index": j, "total # chunks": len(splitted_text)}
print(len(new_texts))
texts = new_texts

[28, 36, 62, 95, 99, 104, 108, 120, 128, 133, 151, 157, 175, 184, 194, 201, 201, 208, 221, 284, 310, 320, 320, 324, 339, 357, 362, 363, 367, 381, 383, 397, 413, 448, 451, 508, 554, 558, 572, 653, 685, 768, 788, 799, 808, 816, 836, 880, 913, 954, 1023, 1154, 1227, 1457, 1490, 2871, 3900, 6775, 12061, 16350]
430


In [20]:
word_frequency = dict()
for text in texts:
    for word in text:
        if word not in word_frequency:
            word_frequency[word] = 0
        word_frequency[word] += 1
word_freq_pairs = [(key, val) for key, val in word_frequency.items()]
word_freq_pairs.sort(key=lambda var: var[1], reverse=True)
word_freq_pairs

[('事', 871),
 ('話', 535),
 ('錢', 370),
 ('手', 353),
 ('心', 348),
 ('船', 327),
 ('家', 318),
 ('頭', 264),
 ('地方', 249),
 ('門', 249),
 ('中國', 242),
 ('島', 216),
 ('國', 212),
 ('天', 211),
 ('海', 205),
 ('聲', 199),
 ('書', 189),
 ('水', 187),
 ('王', 185),
 ('魚', 182),
 ('舟', 179),
 ('衆人', 173),
 ('子', 172),
 ('球', 168),
 ('山', 166),
 ('時', 166),
 ('處', 165),
 ('身', 163),
 ('字', 162),
 ('客', 156),
 ('酒', 150),
 ('石', 146),
 ('道', 144),
 ('面', 142),
 ('東西', 140),
 ('口', 140),
 ('氣球', 140),
 ('氣', 138),
 ('路', 136),
 ('房', 135),
 ('鐘', 134),
 ('父', 130),
 ('風', 128),
 ('主人', 127),
 ('大王', 127),
 ('土人', 126),
 ('窟', 126),
 ('馬', 125),
 ('衣服', 125),
 ('洞', 122),
 ('言', 121),
 ('老婆', 120),
 ('心裡', 118),
 ('眼', 117),
 ('日', 115),
 ('意', 115),
 ('夫人', 114),
 ('小廝', 114),
 ('物', 113),
 ('兄', 113),
 ('腳', 112),
 ('夜', 112),
 ('女', 110),
 ('命', 109),
 ('兵', 105),
 ('月', 104),
 ('女子', 104),
 ('老爺', 104),
 ('今日', 103),
 ('信', 102),
 ('室', 99),
 ('語', 99),
 ('師爺', 99),
 ('名', 98),
 ('法子', 97),
 ('衣', 96),


In [21]:
import gensim
from gensim import corpora
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import warnings
warnings.filterwarnings('ignore')  # To ignore all warnings that arise here to enhance clarity
 
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel

In [26]:
dictionary = gensim.corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(abc) for abc in texts]

In [ ]:
def coherence(num_topics):
    ldamodel = LdaModel(corpus, num_topics=num_topics, id2word = dictionary, passes=40,random_state = 1, iterations=1000, chunksize = 1000,eval_every = None)
    print(ldamodel.print_topics(num_topics=num_topics, num_words=10))
    ldacm = CoherenceModel(model=ldamodel, texts=texts, dictionary=dictionary, coherence='c_v')
    print(ldacm.get_coherence())
    return ldacm.get_coherence()

In [ ]:
x = range(5,11)
# z = [perplexity(i) for i in x]  #如果想用困惑度就选这个
y = [coherence(i) for i in x]
plt.plot(x, y)
plt.xlabel('主题数目')
plt.ylabel('coherence大小')
plt.rcParams['font.sans-serif']=['SimHei']
matplotlib.rcParams['axes.unicode_minus']=False
plt.title('主题-coherence变化情况')
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from pandas.plotting import table

# Sample DataFrame
df = pd.DataFrame({
    'Name': x,
    'Age': y,
    
})

# Create a subplot without frame
fig, ax = plt.subplots(figsize=(8, 3))  # Adjust the figsize to fit your needs
ax.axis('tight')
ax.axis('off')

# Create the table and adjust styling
table(ax, df, loc='center', cellLoc='center', rowLoc='center')

plt.show()

In [ ]:
def perplexity(num_topics):
    ldamodel = LdaModel(corpus, num_topics=num_topics, id2word = dictionary, passes=30)
    print(ldamodel.print_topics(num_topics=num_topics, num_words=10))
    print(ldamodel.log_perplexity(corpus))
    return ldamodel.log_perplexity(corpus)


In [ ]:
x = range(1,21)
z = [perplexity(i) for i in x]  #如果想用困惑度就选这个
#y = [coherence(i) for i in x]
plt.plot(x, z)
plt.xlabel('主题数目')
plt.zlabel('coherence大小')
plt.rcParams['font.sans-serif']=['SimHei']
matplotlib.rcParams['axes.unicode_minus']=False
plt.title('主题-coherence变化情况')
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from pandas.plotting import table

# Sample DataFrame
df = pd.DataFrame({
    'Name': x,
    'Age': z,
    
})

# Create a subplot without frame
fig, ax = plt.subplots(figsize=(8, 3))  # Adjust the figsize to fit your needs
ax.axis('tight')
ax.axis('off')

# Create the table and adjust styling
table(ax, df, loc='center', cellLoc='center', rowLoc='center')

plt.show()

In [28]:
lda = LdaModel(corpus=corpus, id2word=dictionary, num_topics=6, passes=200,random_state = 1, iterations=1000,chunksize = 1000, eval_every = None)
topic_list=lda.print_topics()
print(topic_list)
 
for i in lda.get_document_topics(corpus)[:]:
    listj=[]
    for j in i:
        listj.append(j[1])
    bz=listj.index(max(listj))
    print(i[bz][0])

[(0, '0.017*"事" + 0.013*"錢" + 0.012*"話" + 0.008*"心裡" + 0.008*"大王" + 0.007*"老婆" + 0.006*"師爺" + 0.006*"手" + 0.005*"龍王" + 0.005*"客"'), (1, '0.009*"船" + 0.006*"舟" + 0.005*"海" + 0.005*"主人" + 0.004*"事" + 0.004*"家" + 0.004*"書" + 0.004*"處" + 0.004*"父" + 0.003*"山"'), (2, '0.007*"島" + 0.007*"船" + 0.007*"事" + 0.006*"魚" + 0.005*"兵" + 0.004*"處" + 0.003*"心" + 0.003*"土人" + 0.003*"馬" + 0.003*"牛"'), (3, '0.007*"舟" + 0.006*"國" + 0.006*"事" + 0.005*"中國" + 0.004*"衆人" + 0.004*"海" + 0.004*"心" + 0.004*"島" + 0.003*"洞" + 0.003*"女子"'), (4, '0.019*"事" + 0.016*"話" + 0.011*"家" + 0.010*"錢" + 0.008*"心" + 0.006*"頭" + 0.006*"兵士" + 0.006*"男" + 0.005*"地方" + 0.005*"書"'), (5, '0.008*"事" + 0.008*"手" + 0.007*"球" + 0.007*"氣球" + 0.007*"中國" + 0.006*"小廝" + 0.005*"船" + 0.005*"話" + 0.005*"鐘" + 0.005*"老爺"')]
1
3
3
2
0
2
0
1
3
3
3
4
2
2
3
1
3
4
3
3
2
2
2
2
3
1
2
0
1
1
3
5
1
5
5
5
0
3
3
0
3
4
5
2
5
5
5
5
5
3
4
1
4
0
1
3
0
0
0
2
0
4
4
1
1
0
3
3
3
2
5
5
2
2
5
3
3
5
5
0
5
0
1
1
1
2
0
3
1
5
0
2
0
1
4
3
3
3
3
0
0
0
0
0
2
3
2
2
1
1
1
5
5
5

In [29]:
import pyLDAvis
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda, corpus, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5     -0.023101  0.155132       1        1  26.696335
0      0.157173 -0.084704       2        1  18.091439
4      0.106024  0.059742       3        1  16.879481
3     -0.062636 -0.029112       4        1  13.381913
2     -0.096361 -0.035625       5        1  13.015447
1     -0.081098 -0.065432       6        1  11.935385, topic_info=     Term        Freq       Total Category  logprob  loglift
834     錢  309.000000  309.000000  Default  30.0000  30.0000
741     話  450.000000  450.000000  Default  29.0000  29.0000
9065   心裡   99.000000   99.000000  Default  28.0000  28.0000
1195   氣球  124.000000  124.000000  Default  27.0000  27.0000
3670   大王  106.000000  106.000000  Default  26.0000  26.0000
...   ...         ...         ...      ...      ...      ...
201     風   19.742947  105.272524   Topic6  -6.0162   0.4519
645     天   21.142793  176.393677   Topic6  -5.9477   0.0042
11     地方   20.334931  209.201602   Topic6  -5.9867  -0.2053
395    衆人   19.950090  145.013818   Topic6  -6.0058   0.1421
241     時   19.783624  139.101320   Topic6  -6.0142   0.1753

[454 rows x 6 columns], token_table=      Topic      Freq Term
term                      
74        4  0.936981   上流
1686      1  0.132968   上海
1686      3  0.824399   上海
1686      4  0.026594   上海
9245      2  0.939098   下晚
...     ...       ...  ...
3852      4  0.358735    鼠
3852      5  0.418524    鼠
3852      6  0.059789    鼠
9171      2  0.939082   龍宮
9172      2  0.987160   龍王

[957 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 1, 5, 4, 3, 2])